# Swipe Data

Seperate table per servery (south, baker, siebel, north, west)

cols: time (15 min intervals), swipes

In [9]:
import pandas as pd
import random

def generate_random(start_time, end_time):
    times = pd.date_range(start=start_time, end=end_time, freq='15min')
    # have a check such that times are between 7:30am to 8:00pm
    times = times[(times.time >= pd.Timestamp('07:30:00').time()) & 
                  (times.time <= pd.Timestamp('20:00:00').time())]

    swipes = [random.randint(0, 20) for _ in range(len(times))]
    
    # Create a DataFrame
    df = pd.DataFrame({
        'time': times,
        'swipes': swipes
    })
    
    return df

south_df = generate_random('2025-01-01 00:00:00', '2025-03-01 03:00:00')
south_df

,time,swipes
0,2025-01-01 07:30:00,13
1,2025-01-01 07:45:00,11
2,2025-01-01 08:00:00,10
3,2025-01-01 08:15:00,11
4,2025-01-01 08:30:00,6
...,...,...
3004,2025-02-28 19:00:00,9
3005,2025-02-28 19:15:00,6
3006,2025-02-28 19:30:00,5
3007,2025-02-28 19:45:00,16
